In [2]:
!pip install git+https://github.com/descriptinc/audiotools

  Cloning https://github.com/descriptinc/audiotools to /tmp/pip-req-build-e5wwmvn6
  Running command git clone --filter=blob:none --quiet https://github.com/descriptinc/audiotools /tmp/pip-req-build-e5wwmvn6
  Resolved https://github.com/descriptinc/audiotools to commit 784f87b7679daee82ee7bb25770d5bd51a977bb6
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import pandas as pd

def filter_and_modify_paths(input_csv_path, output_csv_path):
    # CSV 파일 읽기
    df = pd.read_csv(input_csv_path)

    # 'duration'이 3보다 큰 행들만 필터링
    filtered_df = df[df['duration'] > 3]

    # 'audio_path' 열의 경로 수정
    filtered_df['audio_path'] = filtered_df['audio_path'].str.replace('./data', '/workspace/data')

    # 변경된 데이터를 새로운 CSV 파일로 저장
    filtered_df.to_csv(output_csv_path, index=False)

# 사용 예
input_csv_path = './audio_info.csv'  # 원본 CSV 파일 경로
output_csv_path = './audio_info_upper_3seconds.csv'  # 수정된 데이터를 저장할 경로
filter_and_modify_paths(input_csv_path, output_csv_path)


/tmp/ipykernel_4529/702272245.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['audio_path'] = filtered_df['audio_path'].str.replace('./data', '/workspace/data')


In [3]:
data = pd.read_csv(output_csv_path)

In [14]:
import soundfile as sf
wav_path = "/workspace/data/boom-ui-sounds/boom_ui_1/Arcade_Click_Negative_01.wav"#data.iloc[0]['audio_path']
wav, sr = sf.read(wav_path)

In [20]:
info = sf.info(wav_path)
samplerate = info.samplerate
print(samplerate)

offset = 0
duration = 3

sf.read(wav_path, start=int(offset * samplerate), stop=int((offset + duration) * samplerate))

48000


(array([[ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        ...,
        [ 0.00028813, -0.00058341],
        [-0.00162375,  0.00180459],
        [-0.00022244,  0.00079036]]),
 48000)

In [21]:
from audiotools import AudioSignal
duration = 3
wav = AudioSignal(wav_path, offset=0, duration=duration)

In [22]:
output_path = "./test.wav"
wav.write(output_path)

In [5]:
import pandas as pd
import os
from audiotools import AudioSignal

def process_first_audio_and_save_csv(csv_path, output_folder, output_csv_path):
    # CSV 파일 읽기
    df = pd.read_csv(csv_path)

    # 결과를 저장할 리스트 초기화
    results_df = []

    # 출력 폴더 생성 (이미 존재하는 경우에는 생성하지 않음)
    os.makedirs(output_folder, exist_ok=True)

    # CSV 파일의 모든 audio_path에 대해 처리
    for audio_path in df['audio_path']:
        # 출력 경로 생성
        output_path = os.path.join(output_folder, os.path.basename(audio_path))
        # 결과 리스트에 행 추가
        results_df.append({'output_path': output_path, 'duration': 3})

        # 해당 파일이 이미 존재하지 않는 경우에만 처리
        if not os.path.exists(output_path):
            # 오디오 파일의 첫 3초 추출
            wav = AudioSignal(audio_path, offset=0, duration=3)

            # 추출된 오디오 저장
            wav.write(output_path)

            

    # 결과 리스트를 DataFrame으로 변환하고 CSV 파일로 저장
    results_df = pd.DataFrame(results_df)
    results_df.to_csv(output_csv_path, index=False)

# 사용 예
csv_path = './audio_info_upper_3seconds.csv'  # 원본 CSV 파일 경로
output_folder = '/workspace/data/first_3seconds'  # 저장할 폴더 경로
output_csv_path = './processed_first_3seconds_audio_info.csv'  # 결과 CSV 파일 경로
process_first_audio_and_save_csv(csv_path, output_folder, output_csv_path)

/usr/local/lib/python3.10/dist-packages/audiotools/core/audio_signal.py:601: UserWarning: Audio amplitude > 1 clipped when saving
  warnings.warn("Audio amplitude > 1 clipped when saving")


KeyboardInterrupt: 

In [25]:
import pandas as pd
import os
from audiotools import AudioSignal

def process_last_audio_and_save_csv(csv_path, output_folder, output_csv_path):
    # CSV 파일 읽기
    df = pd.read_csv(csv_path)

    # 결과를 저장할 DataFrame 초기화
    results_df = []

    # 출력 폴더 생성 (이미 존재하는 경우에는 생성하지 않음)
    os.makedirs(output_folder, exist_ok=True)

    # CSV 파일의 모든 audio_path에 대해 처리
    for index, item in df.iterrows():
        audio_path = item['audio_path']
        total_duration = item['duration']

        output_path = os.path.join(output_folder, os.path.basename(audio_path))
        # 결과 리스트에 행 추가
        results_df.append({'output_path': output_path, 'duration': 3})

        # 해당 파일이 이미 존재하지 않는 경우에만 처리
        if not os.path.exists(output_path):
            wav = AudioSignal(audio_path, offset=total_duration-3, duration=3)
            # 추출된 오디오 저장
            wav.write(output_path)


    # 결과 DataFrame을 CSV 파일로 저장
    results_df.to_csv(output_csv_path, index=False)

# 사용 예
csv_path = './audio_info_upper_3seconds.csv'  # 원본 CSV 파일 경로
output_folder = '/workspace/data/last_3seconds'  # 저장할 폴더 경로
output_csv_path = './processed_last_3seconds_audio_info.csv'  # 결과 CSV 파일 경로
process_last_audio_and_save_csv(csv_path, output_folder, output_csv_path)


/usr/local/lib/python3.10/dist-packages/audiotools/core/audio_signal.py:501: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sample_rate = librosa.load(
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '/workspace/data/epidemic-audios/zippers/x11jajsgbq.wav'

In [2]:
import os
directory = '/workspace/data/first_3seconds' 
file_list = [name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))]

In [3]:
len(file_list)

117834

In [4]:
import pandas as pd
csv_path = './audio_info_upper_3seconds.csv'  # 원본 CSV 파일 경로
csv = pd.read_csv(csv_path)
print(len(csv))

164086


In [ ]:
import os
import pandas as pd

def create_csv_from_directory(directory, output_csv_path):
    # 디렉토리 내의 파일들을 나열
    files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

    # 각 파일에 대한 정보를 담을 DataFrame 생성
    df = pd.DataFrame({'audio_path': files, 'duration': [3] * len(files)})

    # DataFrame을 CSV 파일로 저장
    df.to_csv(output_csv_path, index=False)

# 사용 예
directory = '/workspace/data/last_3seconds'  # 파일들이 있는 디렉토리
output_csv_path = './processed_last_3seconds_audio_info.csv'  # 저장할 CSV 파일 경로
create_csv_from_directory(directory, output_csv_path)